In [1]:
import numpy as np

def f(x):
    return np.exp(-0.5*x**2)/np.sqrt(2*np.pi)

def int_a_b(num = 1000,a = 2,b = 5):
    x = a + (b-a)*np.random.rand(num)
    y = f(x)
    return (b-a)*np.mean(y)


In [2]:
int_a_b(10000)

0.022749575648506368

无穷积分之统计计算：

从M处截断：
$\int_{-\infty}^t  = \int_{-\infty}^M +\int_{M}^t$

In [3]:
#计算标准正态分布之分布函数

def CMF_norm(num = 10000,t = 1):
    if(t<0):
        return 1-CMF_norm(num,-t)
    if(t>=0):
        return 0.5+int_a_b(num,a = 0,b = t)  #从0截断

In [4]:
CMF_norm(num = 100000,t = 3)

1.000120586283249

hit-or-miss 方法

$Z\sim N(0,1)$,做N次试验

$\Phi_{estimated} = \frac{1}{N} \Sigma I(Z_i \leqslant t)$

In [5]:
n = 10000
z = np.random.normal(size=[n,1])
def int_hitormiss(t = 3):
    return np.sum(z<=t)/n

int_hitormiss(3)

0.9997

方差缩减技术

- Method 1：增加样本量N
- Method 2：重要抽样法：

计算$J = \int_A f(x) \mathrm{d} x$

Suppose $X\sim g(x),Y = \frac{f(X)}{g(X)}$

$\int f(x)dx = \int \frac{f(x)}{g(x)} g(x) dx = E (Y|X)$

then

$\int_{estimated} = E(Y) =\frac{1}{N}\Sigma Y_i =\frac{1}{N} \Sigma \frac{f(X_i)}{g(X_i)}$

- $Var(est) = Var(Y)/N$
- Y接近常数时，方差较小
- t.t.s,g应与f足够接近
- 应同时保证g容易抽取

计算$\int_0^1 e^x \mathrm{d} x $

考虑到其泰勒展开
选$g(x) = \frac{2}{3} (1+x)$

In [6]:
num = 10000
r = np.random.rand(num)
x = np.sqrt(3*r+1)-1  #逆变换产生g
y = np.exp(x)/(2/3*(1+x))
np.mean(y)

1.7166165420823574

使用不同的重要函数估计积分，并比较效率

$$
\int_0^1 \frac{e^{-x}}{1+x^2} \mathrm{d}x
$$

In [7]:
def f(x):
    return np.exp(-x)/(1+x**2)

比较效率：通过计算条件方差进行

- $g_1(x) = 1,0<x<1$

In [60]:
def est_g1(num = 10000):
    x = np.random.rand(num)
    y = f(x)/1
    return np.mean(y)
est_g1()

0.5216823879545027

In [61]:
def est(times = 1000):
    num = 10000
    estm = []
    for i in range(times):
        estm.append(est_g1(10000))
    return estm 
estimates = est(1000)
np.var(estimates)
#np.var(y)等价于np.mean(y**2)-np.mean(y)**2

6.19725826199129e-06

- $g_2(x) = e^{-x},0<x<+\infty$

截取出$[0,1]$上的部分，其余部分对应的$f(x)$应取0，则有

$\int_0^1 f(x) \mathrm{d} x = \int_0^{+\infty} f(x) \mathrm{d}x = \int_0^{+\infty} \frac{f(x)}{g_2(x)} g_2(x) \mathrm{d} x = E(\frac{f(X)}{g_2(X)})$

In [62]:
def f2(x):
    y=[]
    for xi in x:
        if(xi>1):
            y.append(0)
        else:
            y.append(f(xi))
    return y
def est_g2(num = 10000):
    r = np.random.rand(num)
    x = -np.log(1-r)
    y = f2(x)/np.exp(-x)
    return np.mean(y)
est_g2()

0.5230342164745109

In [63]:
def est(times = 1000):
    num = 10000
    estm = []
    for i in range(times):
        estm.append(est_g2(10000))
    return estm 
estimates = est(1000)
np.var(estimates)

1.727962449519651e-05

- $g_3(x) = \frac{1}{\pi(1+x^2)},-\infty<x<+\infty$

做法同前

In [65]:
def f3(x):
    y=[]
    for xi in x:
        if((xi<=1)&(xi>=0)):
            y.append(f(xi))
        else:
            y.append(0)
    return y

#逆变换法生成g3
r = np.random.rand(num)
x = np.tan(np.pi*(r-0.5))
y_inv = f3(x)*(np.pi*(1+x**2))
print(np.mean(y_inv))

#变换抽样生成g3
def est_g3(num = 10000):
    r = -np.pi/2+np.pi*np.random.rand(num)
    x = np.tan(r)
    y_trans = f3(x)*(np.pi*(1+x**2))
    return np.mean(y_trans)
est_g3(10000)

0.5462544670658748


0.5262395017645295

In [66]:
def est(times = 1000):
    num = 10000
    estm = []
    for i in range(times):
        estm.append(est_g3(10000))
    return estm 
estimates = est(1000)
np.var(estimates)

8.738997413162031e-05

- $ g_4(x) = \frac{e^{-x}}{1-e^{-1}},0<x<1 $

In [67]:
def est_g4(num = 10000):
    r = np.random.rand(num)
    x = -np.log(1-r*(1-1/np.e))
    y = f(x)/(np.exp(-x)/(1-1/np.e))
    return np.mean(y)
est_g4()

0.5249147320860077

In [68]:
def est(times = 1000):
    num = 10000
    estm = []
    for i in range(times):
        estm.append(est_g4(10000))
    return estm 
estimates = est(1000)
np.var(estimates)

9.487686256498726e-07

- $g_5(x) = \frac{4}{\pi (1+x^2)},0<x<1$

In [69]:
def est_g5(num = 10000):
    r = np.random.rand(num)
    x = np.tan(np.pi*(r/4))
    y = f(x)*(np.pi*(1+x**2))/4
    return np.mean(y)
est_g5()

0.5247016697272776

In [70]:
def est(times = 1000):
    num = 10000
    estm = []
    for i in range(times):
        estm.append(est_g5(10000))
    return estm 
estimates = est(1000)
np.var(estimates)

1.8236990962378281e-06

In [50]:
#变换抽样法
r = np.pi/4*np.random.rand(num)
x = np.tan(r)
y = f(x)*(np.pi*(1+x**2))/4
np.mean(y)

0.5249198289624268

In [51]:
np.sqrt(np.var(y)/num)

0.0014089069117289178